In [ ]:
from translator import rdf_to_puml

if __name__ == "__main__":
    input_rdf = "../sample/object-graph-1.rdf"
    output_puml = "output1.puml"
    rdf_to_puml(input_rdf, output_puml)

In [ ]:
from owlready2 import get_ontology
from translator import axiom_to_puml
# Example Usage: [ProcuringBusinessProcess,MaterialArtifact, PieceOfEquipment], [MeasurementInformationContentEntity]
input = "GainOfRole"
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
# axiom_result = axiom_to_puml(iofonto["ProcuringBusinessProcess"])
axiom_result = axiom_to_puml(iofonto[input])
print(axiom_result)